<a href="https://colab.research.google.com/github/aengdahly/WebscrapingPaginas/blob/main/WebscrapingImagenGoogle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Script para descargar imágenes de Google.

Código principal y créditos para el webscraping de las imágenes de google  https://github.com/Ladvien/deep_arcane 

Al código anterior se le adicionó:
1. Guardar las imágenes en google drive.
2. Utilizar selenium en colab.
3. Guardar en un excel el nombre del archivo, url y la breve descripción de la imagen


## Librerías.

In [ ]:

import json
import time
import os
import urllib
import os
import time
import requests
import io
import hashlib

from PIL import Image


import signal

import pandas as pd

!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

wd = webdriver.Chrome('chromedriver',options=options)


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
ruta='/content/gdrive/MyDrive/picture/'
search_terms =['"zorzal"']

In [ ]:
number_of_images = 400
GET_IMAGE_TIMEOUT = 5
SLEEP_BETWEEN_INTERACTIONS = 0.1
SLEEP_BEFORE_MORE = 2

In [ ]:
##########
# Scrap
##########


wd.get("https://google.com")


# Credit:
# https://stackoverflow.com/a/22348885
class timeout:
    def __init__(self, seconds=1, error_message="Timeout"):
        self.seconds = seconds
        self.error_message = error_message

    def handle_timeout(self, signum, frame):
        raise TimeoutError(self.error_message)

    def __enter__(self):
        signal.signal(signal.SIGALRM, self.handle_timeout)
        signal.alarm(self.seconds)

    def __exit__(self, type, value, traceback):
        signal.alarm(0)


def fetch_image_urls(
    query: str,
    max_links_to_fetch: int,
    wd: webdriver,
    sleep_between_interactions: int = 1,
):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)

    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = []
    image_texts=[]
    link_page=[]
    link_title=[]
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.rg_i.Q4LuWd")
        number_results = len(thumbnail_results)

        print(
            f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}"
        )

        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls
            actual_images = wd.find_elements_by_css_selector("img.n3VNCb")
            for actual_image in actual_images:
                if actual_image.get_attribute(
                    "src"
                ) and "http" in actual_image.get_attribute("src"):
                    image_urls.append(actual_image.get_attribute("src"))
                    if actual_image.get_attribute("alt"):
                      image_texts.append(actual_image.get_attribute("alt")) 
                    else: image_texts.append('Sin texto')

            image_count = len(image_urls)

            actual_link =wd.find_elements_by_css_selector("a.Beeb4e")
            for link in actual_link:
                if link.get_attribute(
                    "href"
                ) and "http" in link.get_attribute("href"):
                    link_page.append(link.get_attribute("href"))
                    if link.get_attribute("title"):
                      link_title.append(link.get_attribute("title")) 
                    else: link_title.append('Sin texto')


            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(SLEEP_BEFORE_MORE)

            not_what_you_want_button = ""
            try:
                not_what_you_want_button = wd.find_element_by_css_selector(".r0zKGf")
            except:
                pass

            # If there are no more images return.
            if not_what_you_want_button:
                print("No more images available.")
                return image_urls, image_texts

            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button and not not_what_you_want_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls,image_texts, link_page, link_title


def persist_image(folder_path: str, url: str):
    try:
        print("Getting image")
        with timeout(GET_IMAGE_TIMEOUT):
            image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert("RGB")
        file_path = os.path.join(
            folder_path, hashlib.sha1(image_content).hexdigest()[:10] + ".jpg"
        )
        with open(file_path, "wb") as f:
            image.save(f, "JPEG", quality=100)
        print(f"SUCCESS - saved {url} - as {file_path}")
        info=f"SUCCESS - saved {url} - as {file_path}"
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")
        info=f"ERROR - Could not save {url} - {e}"
    return(info)


def search_and_download(search_term: str, target_path="./images", number_images=5):
    target_folder = os.path.join(target_path, "_".join(search_term.lower().split(" ")))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome('chromedriver',options=options) as wd:
        res,texts, link, title = fetch_image_urls(
            search_term,
            number_images,
            wd=wd,
            sleep_between_interactions=SLEEP_BETWEEN_INTERACTIONS,
        )

        rest=[]
        if res is not None:
            for elem in res:
                rest.append(persist_image(target_folder, elem))
        else:
            print(f"Failed to return links for term: {search_term}")
    return res,texts,rest, link, title


for term in search_terms:
    info=search_and_download(term, ruta+term, number_of_images)
    archivo=pd.DataFrame({'Pag_imagen':info[0],'Titulo':info[1],'Descarga':info[2], 'Archivo':[i.split('/')[-1] for i in info[2]]})
    archivo2=pd.DataFrame({'Link':info[3],'Titulo':info[4]})
    archivo.to_excel(ruta+term+'.xlsx')
    archivo2.to_excel(ruta+term+'_2.xlsx')